In [1]:
import pygame
import sys
import random
import os



pygame 2.5.2 (SDL 2.28.3, Python 3.11.8)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
import os

# Get the current working directory
current_dir = os.getcwd()
pieces_folder = os.path.join(current_dir, 'Pieces')

def get_piece_image_path(piece):
    return os.path.join(pieces_folder, f"{piece}.png")

starting_board = [
    ["r", "n", "b", "q", "k", "b", "n", "r"],
    ["p", "p", "p", "p", "p", "p", "p", "p"],
    [" ", " ", " ", " ", " ", " ", " ", " "],
    [" ", " ", " ", " ", " ", " ", " ", " "],
    [" ", " ", " ", " ", " ", " ", " ", " "],
    [" ", " ", " ", " ", " ", " ", " ", " "],
    ["BP", "BP", "BP", "BP", "BP", "BP", "BP", "BP"],
    ["BR", "BN", "BB", "BQ", "BK", "BB", "BN", "BR"]
]

board = [row[:] for row in starting_board]

WIDTH, HEIGHT = 512, 512
DIMENSION = 8
SQUARE_SIZE = HEIGHT // DIMENSION


LIGHT_SQUARE = (240, 217, 181)
DARK_SQUARE = (181, 136, 99)

def draw_board(screen):
    for row in range(DIMENSION):
        for col in range(DIMENSION):
            color = LIGHT_SQUARE if (row + col) % 2 == 0 else DARK_SQUARE
            pygame.draw.rect(screen, color, pygame.Rect(col * SQUARE_SIZE, row * SQUARE_SIZE, SQUARE_SIZE, SQUARE_SIZE))

def place_piece(screen, piece_image_file, row, col):
    full_path = os.path.join(pieces_folder, piece_image_file)
    img = pygame.image.load(full_path)
    img = pygame.transform.scale(img, (SQUARE_SIZE, SQUARE_SIZE))
    screen.blit(img, pygame.Rect(col * SQUARE_SIZE, (7-row) * SQUARE_SIZE, SQUARE_SIZE, SQUARE_SIZE))


--------------------------------------------------------------------------------------------------------Pawn's Movenement Rules-------------------------------------------------------------

In [3]:
def is_square_attacked(board, row, col, player):
    for i in range(8):
        if board[row][i] != ' ' and board[row][i].lower() != player:
            pygame.display.flip()
            return True
        if board[i][col] != ' ' and board[i][col].lower() != player:
            pygame.display.flip()
            return True

    for i in range(1, 8):
        if row - i >= 0 and col + i < 8:
            if board[row - i][col + i] != ' ' and board[row - i][col + i].lower() != player:
                pygame.display.flip()
                return True
        if row - i >= 0 and col - i >= 0:
            if board[row - i][col - i] != ' ' and board[row - i][col - i].lower() != player:
                pygame.display.flip()
                return True
        if row + i < 8 and col + i < 8:
            if board[row + i][col + i] != ' ' and board[row + i][col + i].lower() != player:
                pygame.display.flip()
                return True
        if row + i < 8 and col - i >= 0:
            if board[row + i][col - i] != ' ' and board[row + i][col - i].lower() != player:
                pygame.display.flip()
                return True

    knight_moves = [(2, 1), (2, -1), (-2, 1), (-2, -1), (1, 2), (-1, 2), (1, -2), (-1, -2)]
    for dr, dc in knight_moves:
        if 0 <= row + dr < 8 and 0 <= col + dc < 8:
            if board[row + dr][col + dc].lower() == 'n' and board[row + dr][col + dc].islower() != player.islower():
                pygame.display.flip()
                return True

    rook_directions = [(0, 1), (0, -1), (1, 0), (-1, 0)]
    for dr, dc in rook_directions:
        r, c = row, col
        while 0 <= r + dr < 8 and 0 <= c + dc < 8:
            r += dr
            c += dc
            if board[r][c] != ' ':
                if board[r][c].lower() == 'r' and board[r][c].islower() != player.islower():
                    pygame.display.flip()
                    return True
                else:
                    break

    bishop_directions = [(1, 1), (-1, 1), (1, -1), (-1, -1)]
    for dr, dc in bishop_directions:
        r, c = row, col
        while 0 <= r + dr < 8 and 0 <= c + dc < 8:
            r += dr
            c += dc
            if board[r][c] != ' ':
                if board[r][c].lower() == 'b' and board[r][c].islower() != player.islower():
                    return True
                else:
                    break
                
    king_moves = [(1, 1), (1, 0), (1, -1), (0, 1), (0, -1), (-1, 1), (-1, 0), (-1, -1)]
    for dr, dc in king_moves:
        if 0 <= row + dr < 8 and 0 <= col + dc < 8:
            if board[row + dr][col + dc].lower() == 'k' and board[row + dr][col + dc].islower() != player.islower():
                return True

    return False

In [4]:
from ValidityCheck import (
    is_invalid_move,
    is_valid_pawn_move,
    is_valid_rook_move,
    is_valid_knight_move,
    is_valid_bishop_move,
    is_valid_queen_move,
    is_valid_king_move,
    is_valid_castling
)

def is_valid_move(board, start_row, start_col, end_row, end_col, player, en_passant_possible):
    if is_invalid_move(board, start_row, start_col, end_row, end_col, player):
        print(f"Invalid move detected for {player} from ({start_row}, {start_col}) to ({end_row}, {end_col})")
        return False

    piece = board[start_row][start_col]
    print(f"Attempting to move piece: {piece}")
    
    if (player == 'w' and not piece.isupper()) or (player == 'b' and not piece.islower()):
        print(f"This piece does not belong to player {player}")
        return False

    piece = piece[1:] if piece.startswith('B') else piece
    
    if piece.lower() == 'p':
        return is_valid_pawn_move(board, start_row, start_col, end_row, end_col, player, en_passant_possible)
    elif piece.lower() == 'r':
        return is_valid_rook_move(board, start_row, start_col, end_row, end_col)
    elif piece.lower() == 'n':
        return is_valid_knight_move(start_row, start_col, end_row, end_col)
    elif piece.lower() == 'b':
        return is_valid_bishop_move(board, start_row, start_col, end_row, end_col)
    elif piece.lower() == 'q':
        return is_valid_queen_move(board, start_row, start_col, end_row, end_col)
    elif piece.lower() == 'k':
        return is_valid_king_move(start_row, start_col, end_row, end_col) or is_valid_castling(board, start_row, start_col, end_row, end_col, player)
    
    return False


In [5]:
def make_move(board, screen, start_row, start_col, end_row, end_col):
    piece = board[start_row][start_col]
    
    # Rok kontrolü
    if piece.lower() == 'k' and abs(end_col - start_col) == 2:
        # Kısa rok
        if end_col == 6:
            board[start_row][4] = ' ' 
            board[start_row][6] = piece  
            board[start_row][7] = ' ' 
            board[start_row][5] = 'R' if piece.isupper() else 'r'  
        # Uzun rok
        elif end_col == 2:
            board[start_row][4] = ' '  
            board[start_row][2] = piece  
            board[start_row][0] = ' '  
            board[start_row][3] = 'R' if piece.isupper() else 'r'  
    else:
        # Normal hamle
        board[end_row][end_col] = piece
        board[start_row][start_col] = ' '
    
    if is_valid_castling(board, start_row, start_col, end_row, end_col, board[start_row][start_col]):
        if end_col == 6:
            board[start_row][start_col] = ' '
            board[start_row][end_col] = 'K' if board[start_row][start_col].isupper() else 'k'
            if board[start_row][start_col].isupper():
                board[start_row][7] = ' '
                board[start_row][5] = 'R'
            else:
                board[start_row][0] = ' '
                board[start_row][3] = 'r'
        elif end_col == 2:
            board[start_row][start_col] = ' '
            board[start_row][end_col] = 'K' if board[start_row][start_col].isupper() else 'k'
            if board[start_row][start_col].isupper():
                board[start_row][0] = ' '
                board[start_row][3] = 'R'
            else:
                board[start_row][7] = ' '
                board[start_row][5] = 'r'
        return

In [6]:
def get_clicked_square(mouse_pos):
    col = mouse_pos[0] // SQUARE_SIZE
    row = 7 - (mouse_pos[1] // SQUARE_SIZE)
    return row, col

In [7]:
import Pieces

def make_move(board, screen, start_row, start_col, end_row, end_col):
    piece = board[start_row][start_col]
    board[end_row][end_col] = piece
    board[start_row][start_col] = ' '

    if piece.lower() == 'k' and abs(end_col - start_col) == 2:
        if end_col > start_col:
            rook = board[start_row][7]
            board[start_row][7] = ' '
            board[start_row][5] = rook
        else:
            rook = board[start_row][0]
            board[start_row][0] = ' '
            board[start_row][3] = rook

    if piece.lower() == 'p' and (end_row == 0 or end_row == 7):
        board[end_row][end_col] = 'Q' if piece.isupper() else 'q'

    if piece.lower() == 'p' and abs(start_row - end_row) == 2:
        en_passant_possible = (end_row, end_col)
    else:
        en_passant_possible = None

    if piece.lower() == 'p' and start_col != end_col and board[end_row][end_col] == ' ':
        board[start_row][end_col] = ' '

    draw_board(screen)
    for row in range(DIMENSION):
        for col in range(DIMENSION):
            piece = board[row][col]
            if piece != ' ':
                piece_image_file = piece.upper() + ".png"
                place_piece(screen, piece_image_file, row, col)
    pygame.display.flip()

def main():
    current_player = 'b'
    
    pygame.init()
    screen = pygame.display.set_mode((WIDTH, HEIGHT))
    pygame.display.set_caption("Chess")

    clock = pygame.time.Clock()
    screen.fill((255, 255, 255))
    global en_passant_possible
    en_passant_possible = None
    running = True
    selected_piece = None

    print("Initial board state:")
    for row in board:
        print(row)

    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            elif event.type == pygame.MOUSEBUTTONDOWN and event.button == 1:
                mouse_pos = pygame.mouse.get_pos()
                row, col = get_clicked_square(mouse_pos)
                clicked_piece = board[row][col]
                print(f"Clicked square: {row}, {col}")
                print(f"Current player: {current_player}")
                print(f"Piece at clicked square: '{clicked_piece}'")
                
                if selected_piece is None:
                    if clicked_piece != ' ':
                        if (clicked_piece.isupper() and current_player == 'w') or (clicked_piece.islower() and current_player == 'b'):
                            selected_piece = (row, col)
                            print(f"Selected piece: {clicked_piece} at {row}, {col}")
                        else:
                            print(f"This piece ({clicked_piece}) belongs to the other player.")
                    else:
                        print("No piece at this square.")
                else:
                    start_row, start_col = selected_piece
                    end_row, end_col = row, col
                    if is_valid_move(board, start_row, start_col, end_row, end_col, current_player, en_passant_possible):
                        make_move(board, screen, start_row, start_col, end_row, end_col)
                        current_player = 'b' if current_player == 'w' else 'w'
                        print(f"Moved piece from {start_row}, {start_col} to {end_row}, {end_col}")
                        print("Updated board state:")
                        for row in board:
                            print(row)
                    else:
                        print(f"Invalid move from {start_row}, {start_col} to {end_row}, {end_col}")
                    selected_piece = None

        draw_board(screen)
        for row in range(DIMENSION):
            for col in range(DIMENSION):
                piece = board[row][col]
                if piece != ' ':
                    piece_image_file = piece.upper() + ".png"
                    place_piece(screen, piece_image_file, row, col)
        
        if selected_piece:
            row, col = selected_piece
            pygame.draw.rect(screen, (255, 255, 0), pygame.Rect(col * SQUARE_SIZE, (7-row) * SQUARE_SIZE, SQUARE_SIZE, SQUARE_SIZE), 3)

        pygame.display.flip()
        clock.tick(60)

    pygame.quit()
    sys.exit()

if __name__ == "__main__":
    main()

Initial board state:
['r', 'n', 'b', 'q', 'k', 'b', 'n', 'r']
['p', 'p', 'p', 'p', 'p', 'p', 'p', 'p']
[' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ']
[' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ']
[' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ']
[' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ']
['BP', 'BP', 'BP', 'BP', 'BP', 'BP', 'BP', 'BP']
['BR', 'BN', 'BB', 'BQ', 'BK', 'BB', 'BN', 'BR']
Clicked square: 1, 2
Current player: b
Piece at clicked square: 'p'
Selected piece: p at 1, 2
Clicked square: 3, 2
Current player: b
Piece at clicked square: ' '
Attempting to move piece: p
Moved piece from 1, 2 to 3, 2
Updated board state:
['r', 'n', 'b', 'q', 'k', 'b', 'n', 'r']
['p', 'p', ' ', 'p', 'p', 'p', 'p', 'p']
[' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ']
[' ', ' ', 'p', ' ', ' ', ' ', ' ', ' ']
[' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ']
[' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ']
['BP', 'BP', 'BP', 'BP', 'BP', 'BP', 'BP', 'BP']
['BR', 'BN', 'BB', 'BQ', 'BK', 'BB', 'BN', 'BR']
Clicked square: 6, 2
Current player: w
Piece at cl

SystemExit: 

c:\ProgramData\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
